In [1]:
from sqlalchemy import create_engine
from sodapy import Socrata
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

In [2]:
client = Socrata("data.melbourne.vic.gov.au", None)
results = client.get("b2ak-trbp", limit=1000)
raw_df = pd.DataFrame.from_records(results)


In [3]:
clean_df=raw_df.copy()
clean_df[['Date','Time']] = clean_df['date_time'].str.split('T',expand=True)
del clean_df['date_time']
del clean_df['time']
cols=list(clean_df.columns)
cols = [cols[-1]] + cols[:-1]
cols = [cols[-1]] + cols[:-1]
clean_df=clean_df[cols]
clean_df.tail()

,Date,Time,day,hourly_counts,id,mdate,month,sensor_id,sensor_name,year
995,2019-11-02,11:00:00.000,Saturday,50,2888618,2,November,34,Flinders St-Spark La,2019
996,2019-11-02,11:00:00.000,Saturday,128,2888619,2,November,39,Alfred Place,2019
997,2019-11-02,11:00:00.000,Saturday,112,2888620,2,November,37,Lygon St (East),2019
998,2019-11-02,11:00:00.000,Saturday,304,2888621,2,November,40,Lonsdale St-Spring St (West),2019
999,2019-11-02,11:00:00.000,Saturday,145,2888622,2,November,36,Queen St (West),2019


In [6]:
def filter_stats():
    clean_df['Date']=pd.to_datetime(clean_df.Date)
    x=input("Enter start date m/d/y:")
    y=input("Enter End date m/d/y:")
    sdate=pd.to_datetime(x)
    edate=pd.to_datetime(y)
    filter_df=clean_df.loc[(clean_df.Date>=sdate) & (clean_df.Date<=edate)]
    filter_df

In [5]:
root = tk.Tk()

root.title("Pedestrian Data Count")
root.geometry("500x500") # set the root dimensions
root.pack_propagate(False) # tells the root to not let the widgets inside it determine its size.
root.resizable(0, 0) # makes the root window fixed in size.

frame1 = tk.LabelFrame(root, text="City of Melbourne Datasets")
frame1.place(height=250, width=500)

file_frame = tk.LabelFrame(root, text="Data Selection")
file_frame.place(height=100, width=400, rely=0.70, relx=0)

button1 = tk.Button(file_frame, text="Heat Maps", command=lambda: print_stats())
button1.place(rely=0.15, relx=0.75)

button2 = tk.Button(file_frame, text="Clean Data", command=lambda: Clean_stats())
button2.place(rely=0.15, relx=0.50)

button3 = tk.Button(file_frame, text="Load File", command=lambda: Load_stats())
button3.place(rely=0.15, relx=0.25)

button4 = tk.Button(file_frame, text="Expor to SQL", command=lambda: print_stats())
button4.place(rely=0.55, relx=0.75)

button5 = tk.Button(file_frame, text="Export to csv", command=lambda: Clean_stats())
button5.place(rely=0.55, relx=0.50)

button6 = tk.Button(file_frame, text="Filter Dates", command=lambda: Filter_button())
button6.place(rely=0.55, relx=0.25)



tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1, relwidth=1) # set the height and width of the widget to 100% of its container (frame1).

treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview) # command means update the yaxis view of the widget
treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview) # command means update the xaxis view of the widget
tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) # assign the scrollbars to the Treeview Widget
treescrollx.pack(side="bottom", fill="x") # make the scrollbar fill the x axis of the Treeview widget
treescrolly.pack(side="right", fill="y") # make the scrollbar fill the y axis of the Treeview widget


def Load_stats():
    cols = list(raw_df.columns)
    tv1["columns"] = cols
    for i in cols:
        tv1.column(i, anchor="w")
        tv1.heading(i, text=i, anchor='w')
    for index, row in raw_df.iterrows():
        tv1.insert("",0,text=index,values=list(row))

def Clean_stats():
    cols = list(clean_df.columns)
    tv1["columns"] = cols
    for i in cols:
        tv1.column(i, anchor="w")
        tv1.heading(i, text=i, anchor='w')
    for index, row in clean_df.iterrows():
        tv1.insert("",0,text=index,values=list(row))

def filter_stats():
    clean_df['Date']=pd.to_datetime(clean_df.Date)
    x=input("Enter start date m/d/y:")
    y=input("Enter End date m/d/y:")
    sdate=pd.to_datetime(x)
    edate=pd.to_datetime(y)
    filter_df=clean_df.loc[(clean_df.Date>=sdate) & (clean_df.Date<=edate)]
    filter_df

   
    
root.mainloop()

In [21]:
clean_df.to_csv("Clean_Data.csv")